In [6]:
#!pip install cmake #cmake has been installed seperately 
#for the invalid requirement of gym error it was installed in anaconda command prompt using conda install -c conda-forge gym-atari

!pip install 'gym[atari]' scipy

ERROR: Invalid requirement: "'gym[atari]'"


In [28]:
import gym

#Importing the gym environment and taxi v3. Taxi v2 is no longer available. 

env = gym.make("Taxi-v3").env 

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [29]:
env.reset() # reset environment to a new, random state
env.render()

#shows the action space count and state space count 

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [38]:
state = env.encode(4, 2, 2, 3 ) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 451
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


In [42]:
#reward values for current stat 451
env.P[451] 

{0: [(1.0, 451, -1, False)],
 1: [(1.0, 351, -1, False)],
 2: [(1.0, 451, -1, False)],
 3: [(1.0, 431, -1, False)],
 4: [(1.0, 451, -10, False)],
 5: [(1.0, 451, -10, False)]}

# Without Reinforcement learning

In [43]:
env.s = 451  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    #implementation of reward points and penalty points 
    
    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 3800
Penalties incurred: 1219


In [44]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])#.getvalue()) #getvalue showed an error that did not execute the program. Therefore it has been removed
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 3800
State: 475
Action: 5
Reward: 20


In [45]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [46]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

Wall time: 35.8 s


In [47]:
#q table values checking to see if the model is accrurate
q_table[451]

array([ -2.48340054,  -2.48236806,  -2.48622117,  -2.48236806,
        -9.21160026, -10.47294554])

In [48]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.39
Average penalties per episode: 0.0
